# Ingestando embeddings a Pinecone

In [1]:
%%capture
!pip install pinecone
!pip install tiktoken

In [2]:
# import pinecone
from pinecone import Pinecone, ServerlessSpec
import os, getpass
import tiktoken

In [3]:
api_key = getpass.getpass("Ingresa tu API Key de Pinecone : ")

## Creando un index en Pinecone

In [4]:
index_name = "knowledge-base-eliminatorias"
dimension = 1536

# Conectarse con la única región permitida
pc = Pinecone(api_key=api_key)

if index_name not in [idx.name for idx in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",          # ✅ Solo AWS
            region="us-east-1"    # ✅ Solo us-east-1
        )
    )
    print(f"✅ Índice '{index_name}' creado correctamente.")
else:
    print(f"ℹ️ El índice '{index_name}' ya existe.")


✅ Índice 'knowledge-base-eliminatorias' creado correctamente.


## Generando fragmentos de sitios web

![Imagen](https://imgmedia.larepublica.pe/640x371/larepublica/original/2023/10/17/652f24917388f967a923a805.webp)

In [8]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15/15 [langchain-community]ngchain-community]


In [9]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter

loader = WebBaseLoader(
    [
    "https://www.marca.com/co/2023/10/17/652e070f22601d73648b4585.html", 
    "https://hiraoka.com.pe/blog/post/eliminatorias-sudamericanas-mundial-2026-calendario-partidos-y-fechas"
    ]
)
data = loader.load()

#Genera varios fragmentos de 400 tokens
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 500, 
    chunk_overlap = 20
)

docs = text_splitter.split_documents(data)

USER_AGENT environment variable not set, consider setting it to identify your requests.
Created a chunk of size 1077, which is longer than the specified 500
Created a chunk of size 1371, which is longer than the specified 500
Created a chunk of size 3405, which is longer than the specified 500


In [10]:
len(docs), docs

(28,
 [Document(metadata={'source': 'https://www.marca.com/co/2023/10/17/652e070f22601d73648b4585.html', 'title': 'Tabla de posiciones Eliminatorias Mundial 2026: Clasificación y resultados de la sexta fecha en Sudamérica | Marca', 'description': 'Tras unos cuantos meses de parón de selecciones, y con el 21 de abril como última fecha de las Eliminatorias rumbo al Mundial 2026, la selección nacional regresa a los terrenos de ', 'language': 'es-co'}, page_content='Tabla de posiciones Eliminatorias Mundial 2026: Clasificación y resultados de la sexta fecha en Sudamérica | Marca \n\n\n \n\n \n\n\n \n \n\n\n  \n\n\n \n\nEs noticia: Próximo partido Real MadridClasificación Grupo HNorma ocho segundosReal MadridPaíses Bajos - EspańaAsencioJoan GarcíaLamine YamalLakersAl Ain - Juventus dónde verReal Madrid dónde verAlcaraz Munar donde verJuan LebrónBodegaPartidos hoy Mundial de ClubesPC FútbolFerrariPrecio luz hoyClasificación LigaMundial de ClubesFútbol hoy \n\nPortada de Marca\n\n\nEdición se

In [11]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Ingrese la API Key de OpenAI : ")

## Cargando datos a Pinecone

In [13]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.3/734.3 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [openai]2m3/4 [openai]


In [14]:
from langchain.embeddings.openai import OpenAIEmbeddings
from pinecone import Pinecone as PineconeClient
from uuid import uuid4

# Inicializa Pinecone v3
api_key = getpass.getpass("Ingresa tu API Key de Pinecone : ")
index_name = "knowledge-base-eliminatorias"
pc = PineconeClient(api_key=api_key)
index = pc.Index(index_name)

# Embeddings
embedder = OpenAIEmbeddings()
texts = [doc.page_content for doc in docs]
metadatas = [doc.metadata for doc in docs]

# Embed y subir
vectors = []
for i, text in enumerate(texts):
    embedding = embedder.embed_query(text)
    vectors.append({
        "id": str(uuid4()),
        "values": embedding,
        "metadata": metadatas[i] if i < len(metadatas) else {}
    })

index.upsert(vectors=vectors)
print("✅ Documentos insertados en Pinecone.")


✅ Documentos insertados en Pinecone.
